In [1]:
import grid2op

In [2]:
# Make grid2op env
env_name = "l2rpn_case14_sandbox"
env_glop = grid2op.make(env_name, test=True)

C:\Users\Alex Guerra\Anaconda3\lib\site-packages\grid2op\MakeEnv\Make.py:317: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


In [3]:
# Get state
state_glop = env_glop.reset()
print(state_glop)

In [4]:
import gym
import numpy as np
from grid2op.gym_compat import GymEnv

In [6]:
# Convert to gym environment
env_gym = GymEnv(env_glop)
print(f"The \"env_gym\" is a gym environment: {isinstance(env_gym, gym.Env)}")
obs_gym = env_gym.reset()

The "env_gym" is a gym environment: True


In [7]:
# Printing the size of the state/action space
dim_act_space = np.sum([np.sum(env_gym.action_space[el].shape) for el in env_gym.action_space.spaces])
print(f"The size of the action space is : "
      f"{dim_act_space}")
dim_obs_space = np.sum([np.sum(env_gym.observation_space[el].shape).astype(int) 
                        for el in env_gym.observation_space.spaces])
print(f"The size of the observation space is : "
      f"{dim_obs_space}")

The size of the action space is : 160
The size of the observation space is : 438


In [8]:
# Make the action space a toggle based space to reduce by factor of 2
env_gym.action_space = env_gym.action_space.ignore_attr("set_bus").ignore_attr("set_line_status")

new_dim_act_space = np.sum([np.sum(env_gym.action_space[el].shape) for el in env_gym.action_space.spaces])
print(f"The new size of the action space is : {new_dim_act_space}")

The new size of the action space is : 83


In [14]:
from grid2op.gym_compat import ContinuousToDiscreteConverter

In [ ]:
# Convert continuous actions to discrete via "binning"
env_gym.action_space = env_gym.action_space.reencode_space("redispatch", ContinuousToDiscreteConverter(nb_bins=11))

In [15]:
# Print new action space
print(env_gym.action_space)
# Print observation space
print(env_gym.observation_space)

Dict(change_bus:MultiBinary(57), change_line_status:MultiBinary(20), redispatch:MultiDiscrete([11 11  1  1  1 11]))
Dict(_shunt_bus:Box(-2147483648, 2147483647, (1,), int32), _shunt_p:Box(-inf, inf, (1,), float32), _shunt_q:Box(-inf, inf, (1,), float32), _shunt_v:Box(-inf, inf, (1,), float32), a_ex:Box(0.0, inf, (20,), float32), a_or:Box(0.0, inf, (20,), float32), actual_dispatch:Box(-140.0, 140.0, (6,), float32), attention_budget:Box(0.0, inf, (1,), float32), curtailment:Box(0.0, 1.0, (6,), float32), curtailment_limit:Box(0.0, 1.0, (6,), float32), day:Discrete(32), day_of_week:Discrete(8), duration_next_maintenance:Box(-1, 2147483647, (20,), int32), gen_p:Box(-5.410000324249268, 145.4099884033203, (6,), float32), gen_p_before_curtail:Box(-5.410000324249268, 145.4099884033203, (6,), float32), gen_q:Box(-inf, inf, (6,), float32), gen_v:Box(0.0, inf, (6,), float32), hour_of_day:Discrete(24), is_alarm_illegal:Discrete(2), line_status:MultiBinary(20), load_p:Box(-inf, inf, (11,), float32),

In [16]:
# Only keep info about the flow on the powerlines
# rho, gen_p, load_p, and topo_vect
env_gym.observation_space = env_gym.observation_space.keep_only_attr(["rho", "gen_p", "load_p", "topo_vect", 
                                                                      "actual_dispatch"])
new_dim_obs_space = np.sum([np.sum(env_gym.observation_space[el].shape).astype(int) 
                        for el in env_gym.observation_space.spaces])
print(f"The new size of the observation space is : "
      f"{new_dim_obs_space} (it was {dim_obs_space} before!)")

The new size of the observation space is : 100 (it was 438 before!)


In [18]:
from grid2op.gym_compat import ScalerAttrConverter
from gym.spaces import Box

In [19]:
# Scale observation space to be more normalized for more stable learning
ob_space = env_gym.observation_space
ob_space = ob_space.reencode_space("actual_dispatch",
                                   ScalerAttrConverter(substract=0.,
                                                       divide=env_glop.gen_pmax
                                                       )
                                   )
ob_space = ob_space.reencode_space("gen_p",
                                   ScalerAttrConverter(substract=0.,
                                                       divide=env_glop.gen_pmax
                                                       )
                                   )
ob_space = ob_space.reencode_space("load_p",
                                  ScalerAttrConverter(substract=obs_gym["load_p"],
                                                      divide=0.5 * obs_gym["load_p"]
                                                      )
                                  )

# for even more customization, you can use any functions you want !
shape_ = (env_glop.dim_topo, env_glop.dim_topo)
env_gym.observation_space.add_key("connectivity_matrix",
                                  lambda obs: obs.connectivity_matrix(),  # can be any function returning a gym space
                                  Box(shape=shape_,
                                      low=np.zeros(shape_),
                                      high=np.ones(shape_),
                                    )  # this "Box" should represent the return type of the above function
                                  )
env_gym.observation_space = ob_space
env_gym.observation_space

C:\Users\Alex Guerra\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Dict(actual_dispatch:Box(-1.0, 1.0, (6,), float32), gen_p:Box(-0.1352500021457672, 1.1352499723434448, (6,), float32), load_p:Box(-inf, inf, (11,), float32), rho:Box(0.0, inf, (20,), float32), topo_vect:Box(-1, 2, (57,), int32))